# Make a catalog of SEDs

In [1]:
# Imports
from SEDkit import sed, spectrum, synphot, catalog
import astropy.units as q
import numpy as np
import copy
from pkg_resources import resource_filename
from bokeh.io import output_notebook, show
output_notebook()

/Users/jfilippazzo/miniconda3/envs/astroconda/lib/python3.5/site-packages/pysynphot/locations.py:119: UserWarning: Extinction files should be moved to $PYSYN_CDBS/extinction for compatibility with future versions of pysynphot.
  warnings.warn('Extinction files should be moved to '
/Users/jfilippazzo/miniconda3/envs/astroconda/lib/python3.5/site-packages/pysynphot/locations.py:155: UserWarning: Extinction files not found in /grp/hst/cdbs/grid/extinction
  warnings.warn('Extinction files not found in %s' % (extdir,))
/Users/jfilippazzo/miniconda3/envs/astroconda/lib/python3.5/site-packages/pysynphot/refs.py:114: UserWarning: No graph or component tables found; functionality will be SEVERELY crippled. No files found for /grp/hst/cdbs/mtab/*_tmg.fits.
  'functionality will be SEVERELY crippled. ' + str(e))
/Users/jfilippazzo/miniconda3/envs/astroconda/lib/python3.5/site-packages/pysynphot/refs.py:121: UserWarning: No thermal tables found, no thermal calculations can be performed. No files 

Loading BokehJS ...

First we initialize an empty catalog.

In [28]:
# Make an SED catalog
cat = catalog.Catalog()

Now let's make a few SEDs

In [29]:
# Make an SED
s1 = sed.SED(parallax=(1.7474*q.mas, 0.0501*q.mas), spectral_type='L3', name='2MASS 261832886')
s1.add_photometry_file(resource_filename('SEDkit', 'data/L3_photometry.txt'))
cat.add_SED(s1)

Setting radius to (<Quantity 0.09415541971634411 solRad>, <Quantity 0.019640519289763958 solRad>)
Setting age to (<Quantity 6.0 Gyr>, <Quantity 4.0 Gyr>)


In [30]:
# Make 10 other SEDs (I'll just change the parallax)
for i in range(10):
    s2 = copy.copy(s1)
    s2.parallax = abs(np.random.normal(loc=1.7474, scale=0.1))*q.mas, 0.1*q.mas
    s2.name += str(i)
    cat.add_SED(s2)

Then the (abridged) aggregated results look like this:

In [31]:
cat.results[['name','distance','distance_unc','Lbol_sun','Lbol_sun_unc','Teff','Teff_unc']]

name,distance,distance_unc,Lbol_sun,Lbol_sun_unc,Teff,Teff_unc
,pc,pc,,,K,K
object,object,object,object,object,object,object
2MASS 261832886,572.28,16.41,-0.317,0.025,15675.0,1650.0
2MASS 2618328860,561.88,31.57,-0.333,0.049,15532.0,1678.0
2MASS 2618328861,584.42,34.15,-0.299,0.051,15840.0,1716.0
2MASS 2618328862,591.87,35.03,-0.288,0.052,15941.0,1728.0
2MASS 2618328863,517.96,26.83,-0.403,0.045,14912.0,1603.0
2MASS 2618328864,542.83,29.47,-0.363,0.047,15266.0,1645.0
2MASS 2618328865,623.11,38.83,-0.243,0.054,16356.0,1781.0
2MASS 2618328866,574.95,33.06,-0.313,0.05,15711.0,1700.0


We can also plot the results by passing the `plot` method two column names.

In [32]:
show(cat.plot('distance', 'Teff'))

Or plot a CMD! You can also pass the difference of two magnitudes as a string like so.

In [33]:
show(cat.plot('2MASS.J-2MASS.Ks', 'M_2MASS.J'))

We can then retrieve the SED for a particular source like so:

In [34]:
my_sed = cat.get_SED('2MASS 261832886')
fig = my_sed.plot()

The catalog can also be filtered by providing a column name and value...

In [38]:
cat.filter('name', "'2MASS 2618328861'").results

name,age,age_unc,distance,distance_unc,parallax,parallax_unc,radius,radius_unc,spectral_type,spectral_type_unc,SpT,SpT_fit,membership,reddening,fbol,fbol_unc,mbol,mbol_unc,Lbol,Lbol_unc,Lbol_sun,Lbol_sun_unc,Mbol,Mbol_unc,logg,logg_unc,mass,mass_unc,Teff,Teff_unc,Teff_evo,Teff_evo_unc,Teff_bb,SED,SDSS.u,SDSS.u_unc,M_SDSS.u,M_SDSS.u_unc,SDSS.g,SDSS.g_unc,M_SDSS.g,M_SDSS.g_unc,SDSS.r,SDSS.r_unc,M_SDSS.r,M_SDSS.r_unc,SDSS.i,SDSS.i_unc,M_SDSS.i,M_SDSS.i_unc,SDSS.z,SDSS.z_unc,M_SDSS.z,M_SDSS.z_unc,2MASS.J,2MASS.J_unc,M_2MASS.J,M_2MASS.J_unc,2MASS.H,2MASS.H_unc,M_2MASS.H,M_2MASS.H_unc,2MASS.Ks,2MASS.Ks_unc,M_2MASS.Ks,M_2MASS.Ks_unc
,Gyr,Gyr,pc,pc,mas,mas,solRad,solRad,,,,,,,erg / (cm2 s),erg / (cm2 s),,,erg / s,erg / s,,,,,,,solMass,solMass,K,K,K,K,K,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16
2MASS 2618328861,6.0,4.0,584.42,34.15,1.7110839350622178,0.1,0.09415541971634411,0.019640519289763958,73.0,0.5,None,None,None,0,4.710738027543608e-11,5.709534127198549e-13,14.335,0.013,1.9250856085135066e+33,2.2618757582728754e+32,-0.299,0.051,5.501,0.065,4.53,0.05,0.86,0.03,15840.0,1716.0,5284.0,77.0,None,<SEDkit.sed.SED object at 0x12623db00>,24.688,1.3887,15.852,1.3906,22.531,0.13696,13.695,0.151,19.688,0.018997,10.852,0.065979,17.172,0.0050011,8.3359,0.064026,15.219,0.0050011,6.3828,0.064026,12.469,0.026993,3.6328,0.06897,11.586,0.029999,2.75,0.070007,11.055,0.020996,2.2188,0.067017


...or a wildcard search...

In [39]:
cat.filter('name', "*328861").results

name,age,age_unc,distance,distance_unc,parallax,parallax_unc,radius,radius_unc,spectral_type,spectral_type_unc,SpT,SpT_fit,membership,reddening,fbol,fbol_unc,mbol,mbol_unc,Lbol,Lbol_unc,Lbol_sun,Lbol_sun_unc,Mbol,Mbol_unc,logg,logg_unc,mass,mass_unc,Teff,Teff_unc,Teff_evo,Teff_evo_unc,Teff_bb,SED,SDSS.u,SDSS.u_unc,M_SDSS.u,M_SDSS.u_unc,SDSS.g,SDSS.g_unc,M_SDSS.g,M_SDSS.g_unc,SDSS.r,SDSS.r_unc,M_SDSS.r,M_SDSS.r_unc,SDSS.i,SDSS.i_unc,M_SDSS.i,M_SDSS.i_unc,SDSS.z,SDSS.z_unc,M_SDSS.z,M_SDSS.z_unc,2MASS.J,2MASS.J_unc,M_2MASS.J,M_2MASS.J_unc,2MASS.H,2MASS.H_unc,M_2MASS.H,M_2MASS.H_unc,2MASS.Ks,2MASS.Ks_unc,M_2MASS.Ks,M_2MASS.Ks_unc
,Gyr,Gyr,pc,pc,mas,mas,solRad,solRad,,,,,,,erg / (cm2 s),erg / (cm2 s),,,erg / s,erg / s,,,,,,,solMass,solMass,K,K,K,K,K,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16
2MASS 2618328861,6.0,4.0,584.42,34.15,1.7110839350622178,0.1,0.09415541971634411,0.019640519289763958,73.0,0.5,None,None,None,0,4.710738027543608e-11,5.709534127198549e-13,14.335,0.013,1.9250856085135066e+33,2.2618757582728754e+32,-0.299,0.051,5.501,0.065,4.53,0.05,0.86,0.03,15840.0,1716.0,5284.0,77.0,None,<SEDkit.sed.SED object at 0x12623db00>,24.688,1.3887,15.852,1.3906,22.531,0.13696,13.695,0.151,19.688,0.018997,10.852,0.065979,17.172,0.0050011,8.3359,0.064026,15.219,0.0050011,6.3828,0.064026,12.469,0.026993,3.6328,0.06897,11.586,0.029999,2.75,0.070007,11.055,0.020996,2.2188,0.067017


... or an argument with an operator...

In [40]:
cat.filter('logg', '>=4.5').results

name,age,age_unc,distance,distance_unc,parallax,parallax_unc,radius,radius_unc,spectral_type,spectral_type_unc,SpT,SpT_fit,membership,reddening,fbol,fbol_unc,mbol,mbol_unc,Lbol,Lbol_unc,Lbol_sun,Lbol_sun_unc,Mbol,Mbol_unc,logg,logg_unc,mass,mass_unc,Teff,Teff_unc,Teff_evo,Teff_evo_unc,Teff_bb,SED,SDSS.u,SDSS.u_unc,M_SDSS.u,M_SDSS.u_unc,SDSS.g,SDSS.g_unc,M_SDSS.g,M_SDSS.g_unc,SDSS.r,SDSS.r_unc,M_SDSS.r,M_SDSS.r_unc,SDSS.i,SDSS.i_unc,M_SDSS.i,M_SDSS.i_unc,SDSS.z,SDSS.z_unc,M_SDSS.z,M_SDSS.z_unc,2MASS.J,2MASS.J_unc,M_2MASS.J,M_2MASS.J_unc,2MASS.H,2MASS.H_unc,M_2MASS.H,M_2MASS.H_unc,2MASS.Ks,2MASS.Ks_unc,M_2MASS.Ks,M_2MASS.Ks_unc
,Gyr,Gyr,pc,pc,mas,mas,solRad,solRad,,,,,,,erg / (cm2 s),erg / (cm2 s),,,erg / s,erg / s,,,,,,,solMass,solMass,K,K,K,K,K,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16
2MASS 261832886,6.0,4.0,572.28,16.41,1.7474,0.0501,0.09415541971634411,0.019640519289763958,73.0,0.5,None,None,None,0,4.710738027543608e-11,5.709534127198549e-13,14.335,0.013,1.845937720647223e+33,1.0820205056393471e+32,-0.317,0.025,5.547,0.034,4.54,0.04,0.86,0.03,15675.0,1650.0,5259.0,39.0,None,<SEDkit.sed.SED object at 0x12620dc50>,24.688,1.3887,15.898,1.3887,22.531,0.13696,13.742,0.14001,19.688,0.018997,10.898,0.036011,17.172,0.0050011,8.3828,0.032013,15.219,0.0050011,6.4297,0.032013,12.469,0.026993,3.6797,0.040985,11.586,0.029999,2.7969,0.042999,11.055,0.020996,2.2656,0.037994
2MASS 2618328860,6.0,4.0,561.88,31.57,1.7797294491510989,0.1,0.09415541971634411,0.019640519289763958,73.0,0.5,None,None,None,0,4.710738027543608e-11,5.709534127198549e-13,14.335,0.013,1.7794551853012423e+33,2.0112201362413372e+32,-0.333,0.049,5.587,0.062,4.54,0.04,0.85,0.03,15532.0,1678.0,5231.0,79.0,None,<SEDkit.sed.SED object at 0x108395588>,24.688,1.3887,15.938,1.3906,22.531,0.13696,13.781,0.15002,19.688,0.018997,10.938,0.064026,17.172,0.0050011,8.4219,0.061005,15.219,0.0050011,6.4688,0.061005,12.469,0.026993,3.7188,0.067017,11.586,0.029999,2.8359,0.067993,11.055,0.020996,2.3047,0.064026
2MASS 2618328861,6.0,4.0,584.42,34.15,1.7110839350622178,0.1,0.09415541971634411,0.019640519289763958,73.0,0.5,None,None,None,0,4.710738027543608e-11,5.709534127198549e-13,14.335,0.013,1.9250856085135066e+33,2.2618757582728754e+32,-0.299,0.051,5.501,0.065,4.53,0.05,0.86,0.03,15840.0,1716.0,5284.0,77.0,None,<SEDkit.sed.SED object at 0x12623db00>,24.688,1.3887,15.852,1.3906,22.531,0.13696,13.695,0.151,19.688,0.018997,10.852,0.065979,17.172,0.0050011,8.3359,0.064026,15.219,0.0050011,6.3828,0.064026,12.469,0.026993,3.6328,0.06897,11.586,0.029999,2.75,0.070007,11.055,0.020996,2.2188,0.067017
2MASS 2618328862,6.0,4.0,591.87,35.03,1.689553355807249,0.1,0.09415541971634411,0.019640519289763958,73.0,0.5,None,None,None,0,4.710738027543608e-11,5.709534127198549e-13,14.335,0.013,1.974479197685468e+33,2.3494225583452237e+32,-0.288,0.052,5.474,0.066,4.53,0.05,0.86,0.03,15941.0,1728.0,5300.0,77.0,None,<SEDkit.sed.SED object at 0x126287ac8>,24.688,1.3887,15.828,1.3906,22.531,0.13696,13.672,0.151,19.688,0.018997,10.828,0.067017,17.172,0.0050011,8.3125,0.064026,15.219,0.0050011,6.3594,0.064026,12.469,0.026993,3.6094,0.070007,11.586,0.029999,2.7266,0.070984,11.055,0.020996,2.1953,0.067993
2MASS 2618328863,6.0,4.0,517.96,26.83,1.9306672244701708,0.1,0.09415541971634411,0.019640519289763958,73.0,0.5,None,None,None,0,4.710738027543608e-11,5.709534127198549e-13,14.335,0.013,1.5121412190759217e+33,1.5772436183353492e+32,-0.403,0.045,5.764,0.058,4.56,0.04,0.84,0.04,14912.0,1603.0,5149.0,115.0,None,<SEDkit.sed.SED object at 

... or even a tuple of arguments...

In [46]:
cat.filter('M_SDSS.r', ('>10.9','<11')).results

name,age,age_unc,distance,distance_unc,parallax,parallax_unc,radius,radius_unc,spectral_type,spectral_type_unc,SpT,SpT_fit,membership,reddening,fbol,fbol_unc,mbol,mbol_unc,Lbol,Lbol_unc,Lbol_sun,Lbol_sun_unc,Mbol,Mbol_unc,logg,logg_unc,mass,mass_unc,Teff,Teff_unc,Teff_evo,Teff_evo_unc,Teff_bb,SED,SDSS.u,SDSS.u_unc,M_SDSS.u,M_SDSS.u_unc,SDSS.g,SDSS.g_unc,M_SDSS.g,M_SDSS.g_unc,SDSS.r,SDSS.r_unc,M_SDSS.r,M_SDSS.r_unc,SDSS.i,SDSS.i_unc,M_SDSS.i,M_SDSS.i_unc,SDSS.z,SDSS.z_unc,M_SDSS.z,M_SDSS.z_unc,2MASS.J,2MASS.J_unc,M_2MASS.J,M_2MASS.J_unc,2MASS.H,2MASS.H_unc,M_2MASS.H,M_2MASS.H_unc,2MASS.Ks,2MASS.Ks_unc,M_2MASS.Ks,M_2MASS.Ks_unc
,Gyr,Gyr,pc,pc,mas,mas,solRad,solRad,,,,,,,erg / (cm2 s),erg / (cm2 s),,,erg / s,erg / s,,,,,,,solMass,solMass,K,K,K,K,K,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16
2MASS 2618328860,6.0,4.0,561.88,31.57,1.7797294491510989,0.1,0.09415541971634411,0.019640519289763958,73.0,0.5,None,None,None,0,4.710738027543608e-11,5.709534127198549e-13,14.335,0.013,1.7794551853012423e+33,2.0112201362413372e+32,-0.333,0.049,5.587,0.062,4.54,0.04,0.85,0.03,15532.0,1678.0,5231.0,79.0,None,<SEDkit.sed.SED object at 0x108395588>,24.688,1.3887,15.938,1.3906,22.531,0.13696,13.781,0.15002,19.688,0.018997,10.938,0.064026,17.172,0.0050011,8.4219,0.061005,15.219,0.0050011,6.4688,0.061005,12.469,0.026993,3.7188,0.067017,11.586,0.029999,2.8359,0.067993,11.055,0.020996,2.3047,0.064026
2MASS 2618328867,6.0,4.0,568.19,32.28,1.7599767257309573,0.1,0.09415541971634411,0.019640519289763958,73.0,0.5,None,None,None,0,4.710738027543608e-11,5.709534127198549e-13,14.335,0.013,1.8196467221005303e+33,2.0792842580942676e+32,-0.323,0.05,5.563,0.063,4.54,0.04,0.85,0.03,15619.0,1689.0,5247.0,79.0,None,<SEDkit.sed.SED object at 0x120d56e48>,24.688,1.3887,15.914,1.3906,22.531,0.13696,13.758,0.15002,19.688,0.018997,10.914,0.064026,17.172,0.0050011,8.3984,0.062012,15.219,0.0050011,6.4453,0.062012,12.469,0.026993,3.6953,0.067017,11.586,0.029999,2.8125,0.06897,11.055,0.020996,2.2812,0.065002
